In [1]:
import numpy as np
import pandas as pd
import matplotlib as mpl

import matplotlib.pyplot as plt
%matplotlib inline
#%pylab inline

from pylab import *

# load statsmodels
import statsmodels.api as sm

import itertools

from sklearn import metrics

import random
import scipy
import six
from sklearn.metrics import mean_squared_error
from sklearn.cross_validation import train_test_split

import gc
import json
import re

# use seaborn plotting defaults
import sys
sys.path.append('/Users/tatiana/anaconda/pkgs/seaborn-0.6.0-np110py34_0/lib/python3.4/site-packages')
import seaborn as sns; sns.set()

In [2]:
# nlp libraries
import nltk

# only need to download the first time
# done when the download window closes
#nltk.download()
from nltk import word_tokenize

In [3]:
path = r'/Users/tatiana/rest_reviews_1.csv'
reviews = pd.read_csv(path)
reviews['stars_nom'] = ["one" if star == 1 else "two" if star == 2 else "three" if star == 3 
                else "four" if star == 4 else "five" for star in reviews.stars]
reviews.head()

,funny,useful,cool,stars,text,stars_nom
0,3,3,0,3,My sister already warned me to expect the sand...,three
1,1,1,1,3,Our group tried the family plate which has goo...,three
2,0,0,0,5,"Pepe's is a cozy, family ran place that serves...",five
3,0,0,0,5,This is the best sushi place I've ever been to...,five
4,0,1,0,4,"I fell in love with the food, came back twice,...",four


In [4]:
text_cleaned = [re.sub(r'!', ' exclamation', text) for text in reviews['text']]
text_cleaned = [text.lower() for text in text_cleaned]
text_cleaned = [re.sub(r'[^a-z]', ' ', text) for text in text_cleaned]
reviews['text'] = text_cleaned
documents = [word_tokenize(text) for text in reviews.text]

In [6]:
#documents[1]

In [7]:
all_words = list(itertools.chain(*documents))

In [8]:
from nltk.corpus import stopwords
#import enchant

#eng = enchant.Dict("en_US")

freq_words = nltk.FreqDist(w.lower() for w in all_words 
                           if w not in stopwords.words('english'))# & (eng.check(w) == True))
word_features = sorted(w for w in set(freq_words) if freq_words[w] > 700) # 

def document_features(document): # 
    document_words = set(document) # 
    features = {}
    for word in word_features:
        features['contains({})'.format(word)] = (word.lower() in document_words)
    return features

In [9]:
features = pd.DataFrame([document_features(d) for d in documents])
features.head()

,contains(also),contains(always),contains(amazing),contains(another),contains(area),contains(around),contains(asked),contains(atmosphere),contains(awesome),contains(back),...,contains(wait),contains(want),contains(wanted),contains(wasn),contains(way),contains(well),contains(went),contains(wine),contains(worth),contains(would)
0,False,False,False,False,False,False,False,False,False,True,...,False,False,False,False,False,True,False,False,False,True
1,False,False,False,False,True,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,False,True,False,False,False,False,False,False,True,False,...,False,False,False,False,False,False,False,False,False,False
3,False,True,True,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,True,False,True,...,False,True,False,False,False,False,False,False,False,False


In [10]:
work_table = pd.concat([reviews[['funny','useful','cool']], features], axis=1)
work_table.head()

,funny,useful,cool,contains(also),contains(always),contains(amazing),contains(another),contains(area),contains(around),contains(asked),...,contains(wait),contains(want),contains(wanted),contains(wasn),contains(way),contains(well),contains(went),contains(wine),contains(worth),contains(would)
0,3,3,0,False,False,False,False,False,False,False,...,False,False,False,False,False,True,False,False,False,True
1,1,1,1,False,False,False,False,True,False,False,...,False,False,False,False,False,False,False,False,False,False
2,0,0,0,False,True,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,0,0,0,False,True,True,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,0,1,0,False,False,False,False,False,False,False,...,False,True,False,False,False,False,False,False,False,False


In [11]:
work_table['stars_nom'] = ["one" if star == 1 else "two" if star == 2 else "three" if star == 3 
                else "four" if star == 4 else "five" for star in reviews.stars]
#work_table.to_csv('yelp_wt.csv', index=False)

In [12]:
from sklearn.cluster import DBSCAN
from sklearn import metrics
from sklearn.datasets.samples_generator import make_blobs
from sklearn.preprocessing import StandardScaler

In [11]:
def ngrams(input, n):
  #input = input.split(' ')
  output = {}
  for i in range(len(input)-n+1):
    g = ' '.join(input[i:i+n])
    output.setdefault(g, 0)
    output[g] += 1
  return output

In [12]:
bigs = [ngrams(doc, 2) for doc in documents]
#trig = ngrams(documents, 3)
#fourg = ngrams(documents, 4)

In [20]:
tags = [nltk.pos_tag(doc) for doc in documents]
tags[1]

[('our', 'PRP$'),
 ('group', 'NN'),
 ('tried', 'VBD'),
 ('the', 'DT'),
 ('family', 'NN'),
 ('plate', 'NN'),
 ('which', 'WDT'),
 ('has', 'VBZ'),
 ('good', 'JJ'),
 ('variety', 'NN'),
 ('and', 'CC'),
 ('everything', 'NN'),
 ('in', 'IN'),
 ('it', 'PRP'),
 ('had', 'VBD'),
 ('good', 'JJ'),
 ('to', 'TO'),
 ('excellent', 'NN'),
 ('flavor', 'NN'),
 ('the', 'DT'),
 ('presentation', 'NN'),
 ('was', 'VBD'),
 ('nice', 'JJ'),
 ('and', 'CC'),
 ('all', 'DT'),
 ('the', 'DT'),
 ('meats', 'NNS'),
 ('were', 'VBD'),
 ('cooked', 'VBN'),
 ('appropriately', 'RB'),
 ('the', 'DT'),
 ('paella', 'NN'),
 ('was', 'VBD'),
 ('skillfully', 'RB'),
 ('prepared', 'VBN'),
 ('with', 'IN'),
 ('great', 'JJ'),
 ('tasting', 'NN'),
 ('perfectly', 'RB'),
 ('cooked', 'VBD'),
 ('rice', 'NN'),
 ('authentic', 'JJ'),
 ('chorizo', 'NN'),
 ('chunks', 'NNS'),
 ('large', 'JJ'),
 ('pieces', 'NNS'),
 ('of', 'IN'),
 ('shrimp', 'NN'),
 ('and', 'CC'),
 ('chicken', 'VBN'),
 ('all', 'DT'),
 ('seasoned', 'VBN'),
 ('expertly', 'RB'),
 ('in', 'IN'

In [ ]:
from nltk.corpus import brown
from nltk.tag import UnigramTagger
tagger = UnigramTagger(brown.tagged_sents(categories='news')[:500])
sent = ['']
for word, tag in tagger.tag(sent):
    print(word, '->', tag)

In [ ]:
import nltk
from nltk.tag.simplify import simplify_wsj_tag
tagged_sent = nltk.pos_tag(tokens)
simplified = [(word, simplify_wsj_tag(tag)) for word, tag in tagged_sent]

In [ ]:
tagged_sents = brown.tagged_sents(categories='news')
featuresets = []
for tagged_sent in tagged_sents:
    untagged_sent = nltk.tag.untag(tagged_sent)
    for i, (word, tag) in enumerate(tagged_sent):
        featuresets.append( (pos_features(untagged_sent, i), tag) )

size = int(len(featuresets) * 0.1)
train_set, test_set = featuresets[size:], featuresets[:size]
classifier = nltk.NaiveBayesClassifier.train(train_set)

In [ ]:
from collections import Counter
counts = Counter(tag for word,tag in tagged)
counts

In [ ]:
# normalize proportions
total = sum(counts.values())
dict((word, float(count)/total) for word,count in counts.items())

In [ ]:
#words = nltk.word_tokenize(my_text)
my_bigrams = nltk.bigrams(words)
my_trigrams = nltk.trigrams(words)